## Taking a brief look at the fire dataset

In [1]:
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
from datetime import datetime, timedelta
import os
import numpy as np

In [6]:
df = pd.read_csv("dataset/raw_data/fire_archive_M6_96619.csv")
df.sort_values("acq_date").head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-11.8070,142.0583,313.0,1.0,1.0,2019-08-01,56,Terra,MODIS,48,6.3,297.3,6.6,D,0
285,-14.2000,130.6596,324.2,2.0,1.4,2019-08-01,524,Aqua,MODIS,42,6.3,301.8,28.8,D,0
284,-14.3457,129.8308,321.8,1.7,1.3,2019-08-01,524,Aqua,MODIS,57,6.3,301.9,18.7,D,0
283,-14.2476,130.6929,321.2,2.0,1.4,2019-08-01,524,Aqua,MODIS,61,6.3,303.7,21.0,D,0
282,-14.2568,130.7132,325.2,2.0,1.4,2019-08-01,524,Aqua,MODIS,54,6.3,304.4,30.7,D,0


In [7]:
df.sort_values("acq_date").tail()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
35566,-27.4541,149.4632,329.1,1.2,1.1,2019-09-30,407,Aqua,MODIS,70,6.3,315.2,14.2,D,0
35567,-27.4063,151.6380,334.3,1.6,1.3,2019-09-30,407,Aqua,MODIS,72,6.3,314.2,29.9,D,0
35568,-27.4175,151.6407,331.5,1.6,1.3,2019-09-30,407,Aqua,MODIS,64,6.3,317.6,22.2,D,0
35570,-22.4856,150.7192,327.5,1.8,1.3,2019-09-30,408,Aqua,MODIS,0,6.3,299.7,40.2,D,0
36010,-30.8641,121.4995,309.1,1.8,1.3,2019-09-30,1645,Aqua,MODIS,77,6.3,290.9,23.0,N,2


In [8]:
df.shape

(36011, 15)

## Clean the data frame
We only need lat, long, date, and bright-ti4 for our visualization

In [ ]:
df.drop(['scan', 'track', 'acq_time', 'satellite', 'instrument', 'version', 'brightness', 'bright_t31', 'frp', 'type'], axis=1, inplace=True)

max_conf = df['confidence'].max()
min_conf = df['confidence'].min()
print(max_conf)
print(min_conf)

In [ ]:
df.head()

## Split the data into days & scale weights for heatmap

In [ ]:
days = []
for month in range(8, 10):
    for day in range(1, 32):
        daystr = f'2019-0{month}-{day}'
        if day < 10:
            daystr = f'2019-0{month}-0{day}'

        d = df[df["acq_date"] == daystr][["latitude", "longitude", "confidence"]]
        days.append(d)

days.pop()

In [ ]:
print(days[-1])

Prepare the data for the heatmap

In [ ]:
def scale(x, min, max):
    return (x - min) / (max - min)

data = []
for d in days:
    scaled = d.copy()
    scaled['confidence'] = scaled['confidence'].apply(scale, args=(min_conf, max_conf))
    data.append(scaled.values.tolist())

data[0]

Create the time index so that folium knows what date to put on each step

In [ ]:
start_date = datetime(year=2019, month=8, day=1)
time_index = [(start_date + k * timedelta(1)).strftime("%Y-%m-%d") for k in range(len(data))]

Make a heatmap (lol) of brightness over coordinates

In [ ]:
m = folium.Map(
    [-24.0, 140.0],
    zoom_start=4,
)

hm = HeatMapWithTime(data, index=time_index, auto_play=True)
hm.add_to(m)

os.makedirs('graphs/', exist_ok=True)
m.save('graphs/fire_archive_M6_96619.html')